# Set up


## Imports

In [ ]:
%matplotlib inline

In [ ]:
import sys
import os
from pathlib import Path
import math

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context(context="talk", font_scale=1, rc=None)
sns.set_style('whitegrid')

In [ ]:
# Load the "autoreload" extension
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pandas import IndexSlice as idx
import numpy as np

from collections import OrderedDict

from munch import Munch
from tqdm import tqdm

In [ ]:
import crunchers.sklearn_helpers.exploration as expl
import crunchers.statsmodels_helpers.lazy_stats as stats
import crunchers.pandas_helpers.transformations as xform

In [ ]:
import biorep_etl.data.load_recode as loading

In [2]:
"""Functions used in this pipeline."""

import pandas as pd
import numpy as np

# files
# file = "./data/raw/SnapperGIBioreposito_DATA_2017-01-10_1643.csv"

# generate dataframe
# df = pd.read_csv("./data/raw/SnapperGIBioreposito_DATA_2017-01-10_1643.csv")
df = pd.read_csv("../data/raw/SnapperGIBioreposito_DATA_2017-01-10_1643.csv")


print("dataframe read.")

# generate compound key for visit entry
df['vis_id'] = df[['biorepidnumber', 'sample_date']].astype(str).apply(lambda x: ''.join(x), axis=1)
#print(df['vis_id'])
print("visit key generated.")


# generate family and member id for subject entity
#mem_fam = lambda x: pd.Series([i for i in reversed(x.split('.'))])
#mem_fam = df['member_id, family_id'].apply(mem_fam)
#df = pd.concat([df, mem_fam], axis=1)
#print("family and member id generated")

# create subject dataframe
# sub = df.iloc[:, [4, 8, 9, 12 : 33, 35 : 50]]

dataframe read.
visit key generated.


In [11]:
sub = pd.concat([df.iloc[:, [4, 8, 9]],
                 df.iloc[:, 12 : 33],
                 df.iloc[:, 35 : 50]], axis=1)
sub.head()

,biorepidnumber,registryconsent,biorepconsent,ibdtype,otheribddx,family_history,familymember_ibd___1,familymember_ibd___2,familymember_ibd___3,familymember_ibd___4,...,race___6,hispanic_or_latino,neopics_sample,chb_id,neopics,non_bch_pt,referring_hospital,referring_physician,symptom_onset,date_of_diagnosis
0,1004.0,0.0,1.0,2.0,NaN,0.0,0,0,0,0,...,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,433.2,0.0,1.0,NaN,NaN,NaN,0,0,0,0,...,0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,433.2,0.0,1.0,NaN,NaN,NaN,0,0,0,0,...,0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,2.0,NaN,1.0,0,0,0,0,...,0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,2.0,1.0,1.0,2.0,NaN,0.0,0,0,0,0,...,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
vis = pd.concat([df.iloc[:, 213],
                 df.iloc[:, [4,7,1,53,54, 126]],
                 df.iloc[:, 126: 212]], axis=1)

vis.head()

,vis_id,biorepidnumber,sample_date,samplenumber,nsaid,nsaid_doses,hbi_wellbeing,hbi_wellbeing,hbi_abdpain,hbi_liquidstools,...,othermed_type___3,othermed,date_labs,hct,wbc,plt,crp,esr,alb,notes
0,1004.05/23/13,1004.0,5/23/13,5.231301e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Lo ovral,NaN,37.5,8.32,288,0.47,13.0,4.3,NaN
1,433.28/5/14,433.2,8/5/14,8.051400e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,433.28/16/14,433.2,8/16/14,8.161400e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.04/24/13,1.0,4/24/13,4.241300e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,1,INH,NaN,31.4,2.83,233,0.09,19.0,4.3,NaN
4,2.01/15/13,2.0,1/15/13,1.151300e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Ursodeoxycholic acid,NaN,40.2,6.42,310,0.29,13.0,4.7,NaN


In [9]:
df[['record_id', ["completed_by":"other_immunodeficiency"]]].head()

SyntaxError: invalid syntax (<ipython-input-9-47de646fe791>, line 1)

In [7]:
df.columns.values

array(['record_id', 'samplenumber', 'label_on_sample', 'mrn',
       'biorepidnumber', 'completed_by', 'date_crf_completed',
       'sample_date', 'registryconsent', 'biorepconsent',
       'prior_protocol_number', 'ibdyesno', 'ibdtype', 'otheribddx',
       'family_history', 'familymember_ibd___1', 'familymember_ibd___2',
       'familymember_ibd___3', 'familymember_ibd___4',
       'familymember_ibd___5', 'familymember_ibd___6', 'other_relative',
       'ibd_immunodeficiencies', 'other_immunodeficiency', 'controldx',
       'other_diagnosis', 'control_immunodeficiency',
       'otherimmuno_control', 'psc', 'familymembertype',
       'familymembergidx', 'family_otherdiagnosis', 'gender', 'dob', 'age',
       'race___1', 'race___2', 'race___3', 'race___4', 'race___5',
       'race___6', 'hispanic_or_latino', 'neopics_sample', 'chb_id',
       'neopics', 'non_bch_pt', 'referring_hospital',
       'referring_physician', 'symptom_onset', 'date_of_diagnosis',
       'consanguinity', 'oral_

In [ ]:
df[[]]

## Constants

### Get info for converting/moving this file to `reports`

In [ ]:
# %%javascript
# var kernel = IPython.notebook.kernel;
# var thename = window.document.getElementById("notebook_name").innerHTML;
# var command = "IPYNB_NAME = " + "'"+thename+"'";
# kernel.execute(command);

In [ ]:
# rv = Munch()

# rv.rep_id = 'yyy-mm-dd'
# rv.desc = 'blank'
# rv.ipnb = Path('{IPYNB_NAME}.ipynb'.format(IPYNB_NAME=IPYNB_NAME))
# rv.html = '{ipnb}.html'.format(ipnb=rv.ipnb.stem)
# rv.files = '{ipnb}_files'.format(ipnb=rv.ipnb.stem)
# rv.reprt_dir = '../../reports/{IPYNB_NAME}/'.format(IPYNB_NAME=IPYNB_NAME)

### Other Constants

## Paths

In [ ]:
data_ = "../../data/raw/file.csv"

## Functions

## Loading

# The Work

# Convert to HTML report and move to reports folder

In [ ]:
# rv.reprt_dir

In [ ]:
# !jupyter nbconvert --to html_toc $rv.ipnb

In [ ]:
# !mkdir -p $rv.reprt_dir
## !mv -f $rv.html $rv.files $rv.reprt_dir